In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
data=pd.read_csv("churn_modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15721958,Johnson,535,Germany,Male,62,7,60167.63,2,1,1,68906.46,1
1,2,15746867,Miller,672,Spain,Male,27,0,218576.83,1,0,1,164273.87,1
2,3,15731932,Smith,405,France,Female,26,4,207655.69,2,1,1,174758.91,0
3,4,15965838,Smith,389,Spain,Female,71,2,56164.35,4,0,1,137871.55,1
4,5,15859178,Miller,703,France,Female,44,1,99833.15,3,1,1,18923.24,1


In [3]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,535,Germany,Male,62,7,60167.63,2,1,1,68906.46,1
1,672,Spain,Male,27,0,218576.83,1,0,1,164273.87,1
2,405,France,Female,26,4,207655.69,2,1,1,174758.91,0
3,389,Spain,Female,71,2,56164.35,4,0,1,137871.55,1
4,703,France,Female,44,1,99833.15,3,1,1,18923.24,1
...,...,...,...,...,...,...,...,...,...,...,...
995,849,Germany,Male,81,6,149035.66,2,0,1,59876.63,1
996,575,Spain,Male,71,10,184895.29,2,0,0,33158.95,1
997,491,France,Male,89,2,52742.47,1,1,1,106853.84,0
998,641,Germany,Female,66,2,210405.50,3,0,1,121199.57,0


In [4]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'].astype(str))
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,535,Germany,1,62,7,60167.63,2,1,1,68906.46,1
1,672,Spain,1,27,0,218576.83,1,0,1,164273.87,1
2,405,France,0,26,4,207655.69,2,1,1,174758.91,0
3,389,Spain,0,71,2,56164.35,4,0,1,137871.55,1
4,703,France,0,44,1,99833.15,3,1,1,18923.24,1
...,...,...,...,...,...,...,...,...,...,...,...
995,849,Germany,1,81,6,149035.66,2,0,1,59876.63,1
996,575,Spain,1,71,10,184895.29,2,0,0,33158.95,1
997,491,France,1,89,2,52742.47,1,1,1,106853.84,0
998,641,Germany,0,66,2,210405.50,3,0,1,121199.57,0


In [5]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder 



<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1000 stored elements and shape (1000, 3)>

In [6]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoder.toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [8]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
...,...,...,...
995,0.0,1.0,0.0
996,0.0,0.0,1.0
997,1.0,0.0,0.0
998,0.0,1.0,0.0


In [9]:
data = pd.concat(
    [data.drop('Geography', axis=1), geo_encoded_df],
    axis=1
)

data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,535,1,62,7,60167.63,2,1,1,68906.46,1,0.0,1.0,0.0
1,672,1,27,0,218576.83,1,0,1,164273.87,1,0.0,0.0,1.0
2,405,0,26,4,207655.69,2,1,1,174758.91,0,1.0,0.0,0.0
3,389,0,71,2,56164.35,4,0,1,137871.55,1,0.0,0.0,1.0
4,703,0,44,1,99833.15,3,1,1,18923.24,1,1.0,0.0,0.0


In [10]:
import pickle

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [11]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,535,1,62,7,60167.63,2,1,1,68906.46,1,0.0,1.0,0.0
1,672,1,27,0,218576.83,1,0,1,164273.87,1,0.0,0.0,1.0
2,405,0,26,4,207655.69,2,1,1,174758.91,0,1.0,0.0,0.0
3,389,0,71,2,56164.35,4,0,1,137871.55,1,0.0,0.0,1.0
4,703,0,44,1,99833.15,3,1,1,18923.24,1,1.0,0.0,0.0


In [12]:
X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
X_train

array([[-0.66076784,  0.96076892,  0.10995309, ..., -0.73581643,
        -0.67419986,  1.40498317],
       [ 0.28513236, -1.040833  ,  0.87153291, ..., -0.73581643,
        -0.67419986,  1.40498317],
       [ 0.53874328, -1.040833  , -0.46123178, ...,  1.35903462,
        -0.67419986, -0.7117523 ],
       ...,
       [-0.62649609, -1.040833  ,  0.44314426, ..., -0.73581643,
         1.4832397 , -0.7117523 ],
       [-0.0987112 , -1.040833  ,  0.10995309, ..., -0.73581643,
        -0.67419986,  1.40498317],
       [ 0.97742164,  0.96076892, -0.55642926, ..., -0.73581643,
        -0.67419986,  1.40498317]])

In [14]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [16]:
(X_train.shape[1],)

(12,)

In [17]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [20]:
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])



In [21]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="logs/fit/"+datetime.datetime.now().strftime("%Y%M%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [22]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [23]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=10,
    callbacks=[tensorflow_callback,early_stopping_callback]

)

Epoch 1/10
25/25 [==============================] - 0s 4ms/step - loss: 0.7165 - accuracy: 0.4963 - val_loss: 0.6955 - val_accuracy: 0.5000
Epoch 2/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6766 - accuracy: 0.5875 - val_loss: 0.7227 - val_accuracy: 0.5300
Epoch 3/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6623 - accuracy: 0.6112 - val_loss: 0.7147 - val_accuracy: 0.5250
Epoch 4/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6553 - accuracy: 0.6225 - val_loss: 0.6980 - val_accuracy: 0.5400
Epoch 5/10
25/25 [==============================] - 0s 975us/step - loss: 0.6382 - accuracy: 0.6425 - val_loss: 0.7355 - val_accuracy: 0.5750
Epoch 6/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6244 - accuracy: 0.6612 - val_loss: 0.7615 - val_accuracy: 0.5250


In [24]:
model.save('model.hS')

INFO:tensorflow:Assets written to: model.hS/assets


INFO:tensorflow:Assets written to: model.hS/assets


In [25]:
%load_ext tensorboard

In [26]:
%tensorboard --logdir logs/fit20260418-110451

In [27]:
-logdir logs/fit

SyntaxError: invalid syntax (3682561265.py, line 1)

In [28]:
model.save("model.h5")

/Users/khush/Desktop/deep leaning project/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
